# Effect of prominent Macroeconomic factors on the Housing Price Index in the United States

## Load Packages

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

## Load Data

### Purchase only house price index
asdf
### Unemployment rate
asdf
### Consumer price index
asdf
### Inflation rate US
asdf
### Consumer sentiment
asdf
### Producer price index construction machinery manufacturing
asdf
### Average mortgage interest
asdf
### Personal income
asdf
### Moody's seasoned AAA corporate bond yield
asdf
### Home supply
asdf

## Load Data

In [50]:
# define series_id
series_id_dict = {"house_pi": 'HPIPONM226S',
                 "unemp_rate": "UNRATE",
                 "CPI": "USACPIALLMINMEI",
                 "cons_sent": "UMCSENT",
                 "PI_const": "PCU333120333120",
                 "mortgage_int": "MORTGAGE30US",
                 "personal_inc": "PI",
                 "corp_yield": "AAA",
                 "home_supply": "MSACSR"}

# get data folder path
project_name = "housing-price-index-us"
raw_data_path = os.path.join(os.environ["RESEARCH_PATH"], project_name, "data", "raw")

# get filename
observation_start = "2000-01-01"
observation_end = "2022-06-30"

filename_list = []
for id in series_id_dict:
    filename_list.append(f"data_{series_id_dict[id]}_{observation_start}_to_{observation_end}.feather")


# call the function to fetch the data from FRED API
#data = pd.read_feather(os.path.join(raw_data_path, filename_list[0]))
data = pd.read_feather(os.path.join(raw_data_path, filename_list[0])).iloc[:,0:1]
    
# join all data together in a single pd.DataFrame
temp = 0
for key, value in series_id_dict.items():
    data[key] = pd.read_feather(os.path.join(raw_data_path, filename_list[temp])).iloc[:,1:]
    temp += 1

# change data type to float for all columns (but date)
for key, value in series_id_dict.items():
    data[key] = pd.to_numeric(data[key])

data['date'] = pd.to_datetime(data['date'])

        

print(data)

          date  house_pi  unemp_rate         CPI  cons_sent  PI_const  \
0   2000-01-01    137.10         4.0   71.218520      112.0   172.300   
1   2000-02-01    136.90         4.1   71.640431      111.3   172.300   
2   2000-03-01    137.97         4.0   72.231106      107.1   172.400   
3   2000-04-01    138.91         3.8   72.273297      109.2   172.700   
4   2000-05-01    139.49         4.0   72.357679      110.7   172.600   
..         ...       ...         ...         ...        ...       ...   
265 2022-02-01    380.38         3.8  119.702806       62.8   293.693   
266 2022-03-01    386.07         3.6  121.301004       59.4   294.221   
267 2022-04-01    392.33         3.6  121.978170       65.2   302.832   
268 2022-05-01    396.92         3.6  123.322800       58.4   304.699   
269 2022-06-01    397.14         3.6  125.016771       50.0   305.899   

     mortgage_int  personal_inc  corp_yield  home_supply  
0            8.21        8382.6        7.78          4.3  
1    

## EDA

In [51]:
print(data.head())
print('******************************************************************************************')
print(data.shape)
print('******************************************************************************************')
print(data.dtypes)
print('******************************************************************************************')


        date  house_pi  unemp_rate        CPI  cons_sent  PI_const  \
0 2000-01-01    137.10         4.0  71.218520      112.0     172.3   
1 2000-02-01    136.90         4.1  71.640431      111.3     172.3   
2 2000-03-01    137.97         4.0  72.231106      107.1     172.4   
3 2000-04-01    138.91         3.8  72.273297      109.2     172.7   
4 2000-05-01    139.49         4.0  72.357679      110.7     172.6   

   mortgage_int  personal_inc  corp_yield  home_supply  
0          8.21        8382.6        7.78          4.3  
1          8.33        8443.7        7.68          4.3  
2          8.24        8503.7        7.68          4.3  
3          8.15        8542.9        7.64          4.4  
4          8.52        8580.9        7.99          4.4  
******************************************************************************************
(270, 10)
******************************************************************************************
date            datetime64[ns]
house_pi      

In [42]:
type(data['date'][1])

str

In [33]:
test = pd.to_datetime(data['date'])
type(test[2])
#print(test.shape())

KeyError: 'key of type tuple not found and not a MultiIndex'